## File used to analyse the Japanese dataset in terms of EDES parameters

In [1]:
using CSV
using DataFrames

# Load the OGTT data
data_OGTT = CSV.read("C:/Users/20192809/Documents/Data/ohashi_csv_files/ohashi_OGTT.csv", DataFrame)

# load the subject data
data_subject = CSV.read("C:/Users/20192809/Documents/Data/ohashi_csv_files/ohashi_subjectinfo.csv", DataFrame);

### select the OGTT data from the normal glucose tolerant subjects

In [2]:
data_NGT_info = data_subject[data_subject[!, :type] .== "NGT", :]

# Extract the list of IDs from the NGT data
ngt_ids = data_NGT_info[!, :No]

# Filter the OGTT data for rows where the ID matches an NGT ID
data_NGT_OGTT = data_OGTT[in.(data_OGTT[!, :No], Ref(ngt_ids)), :]

# convert glucose values to mmol/L
data_NGT_OGTT[!, 2:6] = data_NGT_OGTT[!, 2:6] ./ 18

# add bodyweight to OGTT data
data_NGT_OGTT = rightjoin(data_NGT_OGTT, data_NGT_info[:, [:No, :BW]], on = :No)

# drop individuals with missing values
data_NGT_OGTT = dropmissing(data_NGT_OGTT)

Row,No,O-PG(0),O-PG(30),O-PG(60),O-PG(90),O-PG(120),O- IRI(0),O-IRI(30),O-IRI(60),O-IRI(90),O-IRI(120),O-CPR(0),O-CPR(30),O-CPR(60),O-CPR(90),O-CPR(120),BW
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,4.88889,7.72222,10.5556,9.05556,7.5,11.0,58.0,110.0,105.0,119.0,2.6,5.4,9.4,10.6,11.9,78.7
2,2,4.33333,6.77778,8.11111,9.5,7.38889,8.0,38.0,18.0,61.0,44.0,1.6,4.7,6.2,7.8,7.6,98.7
3,3,5.38889,10.5556,10.9444,7.05556,6.38889,10.1,65.2,98.3,47.6,30.2,2.5,6.5,9.4,7.6,6.2,70.0
4,4,5.16667,8.77778,6.83333,6.88889,7.22222,4.1,37.1,14.9,18.9,29.8,1.1,4.9,4.2,4.2,6.0,60.0
5,6,4.72222,6.88889,5.16667,5.27778,6.94444,9.2,84.7,67.8,25.2,109.6,2.6,10.1,8.6,5.7,12.0,66.8
6,7,4.83333,6.88889,6.27778,6.16667,6.33333,10.6,28.0,5.8,20.3,25.7,1.8,4.1,4.8,5.4,6.2,58.0
7,8,5.83333,8.16667,9.05556,7.72222,6.0,9.3,33.0,70.8,40.6,19.6,2.4,4.2,7.7,6.8,4.8,65.0
8,9,5.0,8.44444,10.7778,8.66667,7.16667,4.3,45.4,62.8,76.2,66.8,1.7,5.6,8.6,10.2,10.3,65.0
9,10,4.94444,7.22222,6.11111,5.61111,5.27778,8.0,35.2,31.5,26.5,19.0,1.5,5.4,5.7,4.9,4.3,40.0


## Fit the EDES model to the data and calculate the EDES parameters    

In [3]:
using Distributed

n_cores = length(Sys.cpu_info())

if nprocs()-1 < n_cores
    addprocs(n_cores-nprocs()+1, exeflags="--project")
end

@everywhere begin
    using DifferentialEquations, SciMLBase, Random # for solving ODEs
    using DataFrames
    using Optimization, OptimizationOptimJL, LineSearches # for optimization
    using QuasiMonteCarlo # for LHS
    using Trapz
    include("../EDES/including.jl")
end

# fitting hyperparameters
parameters_to_fit = ["k1", "k5", "k6"]
ub = [0.5, 1.5, 10.]
lb =  [0.0005, 0, 0]
save_file = false

# extract data from the OGTT data
glucose =Matrix(data_NGT_OGTT[!, 2:6])
insulin = Matrix(data_NGT_OGTT[!, 7:11])
bodyweight = data_NGT_OGTT[!, :BW]
G_dose = 75000
time_G = [0, 30, 60, 90, 120]
time_I = [0, 30, 60, 90, 120]

# Prepare the patients list with positional and keyword arguments as a tuple
patients = [
    (glucose[i, :], insulin[i, :], bodyweight[i], G_dose, time_G, time_I, 
    (parameters_to_fit, ub, lb, save_file)) 
    for i in 1:size(glucose, 1)
]

# Wrapper function to pass keyword arguments correctly
@everywhere function wrapper(params)
    glucose, insulin, BW, meal_G, time_G, time_I, (estimated_params, ub, lb, save_boolean) = params
    ParameterEstimation(
        glucose,
        insulin,
        BW,
        meal_G,
        time_G,
        time_I;
        estimated_params = estimated_params,
        ub = ub,
        lb = lb,
        save_boolean = save_boolean
    )
end

# Use pmap with the wrapper
results = pmap(x -> wrapper(x), patients)

# close the parallel pool

rmprocs(procs()[2:end])


      From worker 6:	Optimization successful! continuing...
      From worker 6:	Parameter estimation done!
      From worker 2:	Optimization successful! continuing...
      From worker 7:	Optimization successful! continuing...
      From worker 7:	Parameter estimation done!
      From worker 2:	Parameter estimation done!
      From worker 4:	Optimization successful! continuing...
      From worker 4:	Parameter estimation done!
      From worker 8:	Optimization successful! continuing...
      From worker 8:	Parameter estimation done!
      From worker 5:	Optimization successful! continuing...
      From worker 5:	Parameter estimation done!
      From worker 9:	Optimization successful! continuing...
      From worker 9:	Parameter estimation done!
      From worker 6:	Optimization successful! continuing...
      From worker 6:	Parameter estimation done!
      From worker 2:	Optimization successful! continuing...
      From worker 2:	Parameter estimation done!
      From worker 5:	Optimiz

Task (done) @0x0000024b8f01e100

In [4]:
using Statistics

df_k1 = DataFrame(k1 = [results[i][1] for i in 1:length(results)])
df_k5 = DataFrame(k5 = [results[i][2] for i in 1:length(results)])
df_k6 = DataFrame(k6 = [results[i][3] for i in 1:length(results)])

# get statistics of df
# describe(df_k1)

describe(df_k6)

# describe(df_k5)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Float64,Float64,Float64,Int64,DataType
1,k6,2.45694,3.47277e-15,1.90766,9.99962,0,Float64


## visually inspect the fits

In [6]:
using GLMakie

include("./PlotAnalysis.jl")

results_df = hcat(df_k1, df_k5, df_k6)

data_NGT_OGTT
time_G = [0, 30, 60, 90, 120]
time_I = [0, 30, 60, 90, 120]
G_dose = 75000

glucose = Matrix(data_NGT_OGTT[!, 2:6])
insulin = Matrix(data_NGT_OGTT[!, 7:11])
BW = data_NGT_OGTT[!, :BW]

Interactive_plots_EDES(results_df, glucose, insulin, BW, time_G, time_I, G_dose = G_dose)






49-element Vector{Bool}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [ ]:


glucose = Matrix(data_NGT_OGTT[!, 2:6])
insulin = Matrix(data_NGT_OGTT[!, 7:11])
BW = data_NGT_OGTT[!, :BW]


new_df = DataFrame()

